# Power Flow Comparasion

This notebook aims to compare the power flow results from different power flow solvers, namely `pandapower`, `pypower` and `andes`.

In [1]:
import sys
import datetime
import importlib

import numpy as np
import pandas as pd

import andes
import ams

import pandapower as pdp
import pypower.api

Package versions

In [2]:
print("Python version:", sys.version)
print("Last run time:", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

for pkg in ['numba', 'pypower', 'pandapower']:
    print(f'{pkg}: {importlib.metadata.version(pkg)}')

print("andes version:", andes.__version__)
print("ams version:", ams.__version__)

Python version: 3.9.18 | packaged by conda-forge | (main, Aug 30 2023, 03:53:08) 
[Clang 15.0.7 ]
Last run time: 2024-03-22 10:54:40
numba: 0.58.1
pypower: 5.1.16
pandapower: 2.13.1
andes version: 1.9.1.post20+g127dc462
ams version: 0.9.4.post4+g47e0798


## ANDES

In [3]:
andes.config_logger(stream_level=20)

In [4]:
case_xlsx = 'case_ACTIVSg10k.xlsx'
sa = andes.load(case_xlsx,
                setup=True, no_output=True, default_config=True,)

Working directory: "/Users/jinningwang/Documents/work/psal/src/pflow_benchmark"
> Loaded generated Python code in "/Users/jinningwang/.andes/pycode".
Parsing input file "case_ACTIVSg10k.xlsx"...
Input file parsed in 3.2606 seconds.
System internal structure set up in 0.2249 seconds.


In [5]:
sa.PFlow.run(method='NR')

-> System connectivity check results:
  No islanded bus detected.
  System is interconnected.
  Each island has a slack bus correctly defined and enabled.

-> Power flow calculation
           Numba: Off
   Sparse solver: KLU
 Solution method: NR method
Power flow initialized in 0.0089 seconds.
0: |F(x)| = 202.4381598
1: |F(x)| = 4.028044858
2: |F(x)| = 0.1858979964
3: |F(x)| = 0.0002036381987
4: |F(x)| = 4.516184815e-09
Converged in 5 iterations in 0.1248 seconds.


True

## AMS

In [6]:
ams.config_logger(stream_level=20)

In [7]:
sp = ams.load(case_xlsx,
              setup=True, no_output=True, default_config=True)

Working directory: "/Users/jinningwang/Documents/work/psal/src/pflow_benchmark"
Parsing input file "case_ACTIVSg10k.xlsx"...
Input file parsed in 3.4639 seconds.
Zero line rates detacted in rate_a, rate_b, rate_c, adjusted to 999.
If expect a line outage, please set 'u' to 0.
System set up in 0.0225 seconds.


In [8]:
sp.PFlow.run()

<PFlow> initialized in 0.0955 seconds.
 0: |F(x)| = 197.242032
 1: |F(x)| = 4.02839116
 2: |F(x)| = 0.2552148286
 3: |F(x)| = 0.0002035501971
 4: |F(x)| = 4.509343579e-09
<PFlow> solved in 0.1699 seconds, converged in 4 iterations with PYPOWER-Newton.


True

## PYPOWER

In [9]:
ppc = andes.io.matpower.system2mpc(sa)

ppopt = pypower.api.ppoption(VERBOSE=0, OUT_ALL=0, PF_ALG=1)
ppc_sol, _ = pypower.api.runpf(ppc, ppopt)

In [10]:
ppc_bus_cols = ['bus_i', 'type', 'Pd', 'Qd', 'Gs', 'Bs', 'area', 'Vm', 'Va', 'baseKV', 'zone', 'Vmax', 'Vmin']
ppc_bus = pd.DataFrame(ppc_sol['bus'], columns=ppc_bus_cols)

## pandapower

In [11]:
ppn = andes.interop.pandapower.to_pandapower(sa, verify=False)

pdp.runpp(ppn, algorithm='nr')

/Users/jinningwang/Documents/work/andes/andes/interop/pandapower.py:294: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lambda x: x[0] if x[2] >= x[3] else x[1], axis=1)
/Users/jinningwang/Documents/work/andes/andes/interop/pandapower.py:296: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lambda x: x[0] if x[2] < x[3] else x[1], axis=1)
/Users/jinningwang/Documents/work/andes/andes/interop/pandapower.py:299: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, us

## Comparasion

In [12]:
compdf = pd.DataFrame()

# voltage amplitude
bus_idx = sa.Bus.idx.v
compdf['bus'] = bus_idx

compdf['v_andes'] = sa.Bus.get(src='v', attr='v', idx=bus_idx)
compdf['v_ams'] = sp.Bus.get(src='v', attr='v', idx=bus_idx)
compdf['v_pyp'] = ppc_sol['bus'][:, 7]
compdf['v_pdp'] = ppn.res_bus['vm_pu']

# voltage angle
compdf['a_andes'] = sa.Bus.get(src='a', attr='v', idx=bus_idx) * andes.shared.rad2deg
compdf['a_ams'] = sp.Bus.get(src='a', attr='v', idx=bus_idx) * andes.shared.rad2deg
compdf['a_pyp'] = ppc_sol['bus'][:, 8]
compdf['a_pdp'] = ppn.res_bus['va_degree']

# calibrate using slack bus
bus_uid = sa.Bus.idx2uid(sa.Slack.bus.v)[0]
a_cols = ['a_andes', 'a_ams', 'a_pyp', 'a_pdp']
compdf[a_cols] = compdf[a_cols] - compdf[a_cols].iloc[bus_uid]

In [13]:
compdf.head()

,bus,v_andes,v_ams,v_pyp,v_pdp,a_andes,a_ams,a_pyp,a_pdp
0,10001,1.009725,1.009725,1.009725,1.009725,7.900361,7.900997,7.900997,7.900997
1,10002,1.015283,1.015283,1.015283,1.015283,10.704401,10.705036,10.705036,10.705036
2,10003,1.030120,1.030120,1.030120,1.030120,13.711965,13.712598,13.712598,13.712598
3,10004,1.031171,1.031171,1.031171,1.031171,13.824762,13.825394,13.825394,13.825394
4,10005,1.031119,1.031119,1.031119,1.031119,13.818454,13.819087,13.819087,13.819087


Export comparasion to CSV.

In [14]:
compdf.to_csv('comp.csv', index=False)